# Predicting AQI values in Montreal, QC

This notebook contains the code referenced in the original blog post: https://tlbvr.com/blog/predicting-montreal-aqi

In [13]:
import pandas as pd

In [2]:
!curl -o aqi-2022-2024.csv -A "some-user-agent" -L https://donnees.montreal.ca/dataset/547b8052-1710-4d69-8760-beaa3aa35ec6/resource/0c325562-e742-4e8e-8c36-971f3c9e58cd/download/rsqa-indice-qualite-air-2022-2024.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2033  100  2033    0     0   2180      0 --:--:-- --:--:-- --:--:--  2178
100 20.8M  100 20.8M    0     0  5131k      0  0:00:04  0:00:04 --:--:-- 9338k


In [14]:
df = pd.read_csv("./aqi-2022-2024.csv")
df.rename(columns={
    'polluant': 'pollutant',
    'valeur': 'value',
    'date': 'date',
    'heure': 'hour'
}, inplace=True)
df

,stationId,pollutant,value,date,hour
0,103,O3,15,2022-01-15,3
1,103,NO2,2,2022-01-15,3
2,103,PM,12,2022-01-15,3
3,17,CO,1,2022-02-04,21
4,17,O3,17,2022-02-04,21
...,...,...,...,...,...
835003,99,NO2,1,2024-06-25,3
835004,99,PM,16,2024-06-25,3
835005,103,O3,6,2024-06-25,3
835006,103,NO2,3,2024-06-25,3


In [15]:
# Creating a datetime column for easier manipulation later on
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hour'].astype(str) + ':00:00',
                                format = '%Y-%m-%d %H:%M:%S',
                                errors = 'coerce')
# Dropping columns we won't need anymore
df.drop(["hour", "pollutant", "date"], axis=1, inplace=True)

# Indexing by station and datetime and grab the maximum value for each index (related to our assumption earlier)
df = df.groupby(['stationId', 'datetime']).max("value")

df

value
stationId datetime                  
3         2022-01-01 00:00:00     57
          2022-01-01 01:00:00     58
          2022-01-01 02:00:00     60
          2022-01-01 03:00:00     62
          2022-01-01 04:00:00     68
...                              ...
103       2024-06-25 19:00:00     27
          2024-06-25 20:00:00     26
          2024-06-25 21:00:00     23
          2024-06-25 22:00:00     25
          2024-06-25 23:00:00     26

[238490 rows x 1 columns]

In [16]:
df = df.groupby("datetime").mean("value").reset_index()
# Sorting the dataframe by datetime for better visualization
df.sort_values("datetime", inplace=True)
df

,datetime,value
0,2022-01-01 00:00:00,47.818182
1,2022-01-01 01:00:00,53.727273
2,2022-01-01 02:00:00,60.272727
3,2022-01-01 03:00:00,65.454545
4,2022-01-01 04:00:00,65.363636
...,...,...
21763,2024-06-25 19:00:00,27.100000
21764,2024-06-25 20:00:00,26.100000
21765,2024-06-25 21:00:00,23.900000
21766,2024-06-25 22:00:00,24.500000


In [17]:
# This station is located close to the airport and is the one I found holds the most interesting data
station_id="30165"
years = range(2022, 2025)
months = range(1, 13)
# Hourly data
timeframe="1"
dates = [(month, year) for year in years for month in months]
all_files = []

for date in dates:
    month, year = date
    url = f"https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station_id}&Year={year}&Month={month}&Day=1&timeframe={timeframe}&submit=Download+Data"
    all_files.append(pd.read_csv(url))

In [18]:
weather_df = pd.concat(all_files)
weather_df.rename(columns={'Date/Time (LST)': 'datetime', "Temp (°C)": "temp", "Precip. Amount (mm)": "precip", "Rel Hum (%)": "rel_humid"}, inplace=True)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
weather_df = weather_df[['datetime', "temp", "rel_humid", "precip"]]
weather_df

,datetime,temp,rel_humid,precip
0,2022-01-01 00:00:00,0.0,93.0,0.0
1,2022-01-01 01:00:00,0.1,94.0,0.0
2,2022-01-01 02:00:00,0.1,94.0,0.0
3,2022-01-01 03:00:00,0.1,97.0,0.0
4,2022-01-01 04:00:00,-0.5,96.0,0.0
...,...,...,...,...
739,2024-12-31 19:00:00,NaN,NaN,NaN
740,2024-12-31 20:00:00,NaN,NaN,NaN
741,2024-12-31 21:00:00,NaN,NaN,NaN
742,2024-12-31 22:00:00,NaN,NaN,NaN


In [19]:
merged_df = pd.merge(df, weather_df, on="datetime", how="left")
merged_df

,datetime,value,temp,rel_humid,precip
0,2022-01-01 00:00:00,47.818182,0.0,93.0,0.0
1,2022-01-01 01:00:00,53.727273,0.1,94.0,0.0
2,2022-01-01 02:00:00,60.272727,0.1,94.0,0.0
3,2022-01-01 03:00:00,65.454545,0.1,97.0,0.0
4,2022-01-01 04:00:00,65.363636,-0.5,96.0,0.0
...,...,...,...,...,...
21763,2024-06-25 19:00:00,27.100000,25.2,46.0,0.0
21764,2024-06-25 20:00:00,26.100000,23.8,58.0,0.0
21765,2024-06-25 21:00:00,23.900000,23.6,59.0,0.0
21766,2024-06-25 22:00:00,24.500000,19.9,84.0,0.0


In [20]:
merged_df['year'] = merged_df['datetime'].dt.year
# Year has a bigger range than the rest so we divide it up by its maximum to scale it down.
merged_df['year'] = merged_df['year'] / merged_df['year'].max()
merged_df['month'] = merged_df['datetime'].dt.month
merged_df['day'] = merged_df['datetime'].dt.day
merged_df['hour'] = merged_df['datetime'].dt.hour
merged_df['weekday'] = merged_df['datetime'].dt.weekday

# Values above 100 are extreme outliers (and very rare for Montreal). Clamping helps the model not being influenced too much from these rare events.
merged_df['value'] = merged_df['value'].clip(upper=100)
max_value = merged_df['value'].max()
merged_df['value'] = merged_df['value'] / max_value
merged_df

,datetime,value,temp,rel_humid,precip,year,month,day,hour,weekday
0,2022-01-01 00:00:00,0.478182,0.0,93.0,0.0,0.999012,1,1,0,5
1,2022-01-01 01:00:00,0.537273,0.1,94.0,0.0,0.999012,1,1,1,5
2,2022-01-01 02:00:00,0.602727,0.1,94.0,0.0,0.999012,1,1,2,5
3,2022-01-01 03:00:00,0.654545,0.1,97.0,0.0,0.999012,1,1,3,5
4,2022-01-01 04:00:00,0.653636,-0.5,96.0,0.0,0.999012,1,1,4,5
...,...,...,...,...,...,...,...,...,...,...
21763,2024-06-25 19:00:00,0.271000,25.2,46.0,0.0,1.000000,6,25,19,1
21764,2024-06-25 20:00:00,0.261000,23.8,58.0,0.0,1.000000,6,25,20,1
21765,2024-06-25 21:00:00,0.239000,23.6,59.0,0.0,1.000000,6,25,21,1
21766,2024-06-25 22:00:00,0.245000,19.9,84.0,0.0,1.000000,6,25,22,1


In [21]:
merged_df.isna().sum()

datetime      0
value         0
temp         14
rel_humid    14
precip       22
year          0
month         0
day           0
hour          0
weekday       0
dtype: int64

In [22]:
merged_df.fillna({"precip": 0,
                   "temp": merged_df['temp'].fillna(method='bfill'),
                   "rel_humid": merged_df['rel_humid'].mode()[0]}, inplace=True)

assert merged_df[merged_df.isna().any(axis=1)].empty == True

/var/folders/gd/ctbm7cmd6n124vc1h8766rlw0000gn/T/ipykernel_24506/1283295723.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  "temp": merged_df['temp'].fillna(method='bfill'),


In [23]:
import numpy as np

date_valid = pd.Timestamp('2023-01-01')
date_test = pd.Timestamp('2024-01-01')
now = pd.Timestamp('2024-06-01')

# From Jan 01, 2022 -> Dec 31, 2022
train_idx = merged_df['datetime'] < date_valid

# From Jan 01, 2023 -> Dec 31, 2023
valid_idx = (merged_df['datetime'] >= date_valid) & (merged_df['datetime'] < date_test)

# From Jan 01, 2024 -> June 01, 2024
test_idx = (merged_df['datetime'] >= date_test) & (merged_df['datetime'] < now)

train_idxs = np.where(train_idx)[0].tolist()
valid_idxs = np.where(valid_idx)[0].tolist()
test_idxs = np.where(test_idx)[0].tolist()

In [24]:
assert merged_df.iloc[train_idxs[-1]]['datetime'] == pd.Timestamp('2022-12-31 23:00:00')
assert merged_df.iloc[valid_idxs[-1]]['datetime'] == pd.Timestamp('2023-12-31 23:00:00')
assert merged_df.iloc[test_idxs[-1]]['datetime'] == pd.Timestamp('2024-05-31 23:00:00')

In [25]:
from fastai.tabular.all import *

cont,cat = cont_cat_split(merged_df, max_card=20, dep_var='value')
dls = TabularPandas(
  merged_df,
  procs=[Categorify, Normalize],
  cat_names=cat,
  cont_names=cont,
  y_names=['value'],
  splits=(train_idxs, valid_idxs),
  y_block=RegressionBlock()
).dataloaders(bs=2048)

dls.show_batch()

,datetime,month,weekday,temp,rel_humid,precip,year,day,hour,value
0,2022-03-09 08:00:00,3,2,-2.200000,74.0,2.000000e-01,0.999012,9.0,8.0,0.134545
1,2022-01-24 13:00:00,1,0,-13.900000,56.0,4.669100e-09,0.999012,24.0,13.0,0.271818
2,2022-05-31 21:00:00,5,1,17.299999,54.0,4.669100e-09,0.999012,31.0,21.0,0.144545
3,2022-09-30 06:00:00,9,4,6.400000,93.0,4.669100e-09,0.999012,30.0,6.0,0.116364
4,2022-12-03 23:00:00,12,5,1.300000,57.0,4.669100e-09,0.999012,3.0,23.0,0.133636
5,2022-04-22 06:00:00,4,4,6.400000,77.0,4.669100e-09,0.999012,22.0,6.0,0.143636
6,2022-07-21 17:00:00,7,3,28.600001,57.0,4.669100e-09,0.999012,21.0,17.0,0.250909
7,2022-12-15 15:00:00,12,3,4.900000,63.0,4.669100e-09,0.999012,15.0,15.0,0.125455
8,2022-11-27 13:00:00,11,6,9.600000,53.0,4.669100e-09,0.999012,27.0,13.0,0.176364
9,2022-05-28 23:00:00,5,5,17.000000,67.0,4.669100e-09,0.999012,28.0,23.0,0.147273


In [26]:
# In fastai, models are called learner
learn = tabular_learner(dls, metrics=mae, layers=[250, 100], y_range=(0, 1))
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,mae,time
0,0.124582,0.044517,0.189672,00:00
1,0.116288,0.665172,0.810327,00:00
2,0.109973,0.665172,0.810327,00:00
3,0.103871,0.665172,0.810327,00:00
4,0.098426,0.665172,0.810327,00:00
5,0.093807,0.665172,0.810327,00:00
6,0.089975,0.665172,0.810327,00:00
7,0.086668,0.665172,0.810327,00:00
8,0.083967,0.665172,0.810327,00:00
9,0.081800,0.665172,0.810327,00:00


**This isn't training on purpose**. The reason can be found in the blog post.

In [27]:
df_test = merged_df.loc[test_idxs]
test_dl = dls.test_dl(df_test)

preds, targets = learn.get_preds(dl=test_dl)

mae(preds, targets)

TensorBase(0.8181)

In [4]:
!curl -o aqi-2019-2021.csv -A "some-user-agent" -L https://donnees.montreal.ca/dataset/547b8052-1710-4d69-8760-beaa3aa35ec6/resource/e43dc1d6-fbdd-49c3-a79f-83f63404c281/download/rsqa-indice-qualite-air-2019-2021.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2033  100  2033    0     0   1699      0  0:00:01  0:00:01 --:--:--  1701
100 25.5M  100 25.5M    0     0  5353k      0  0:00:04  0:00:04 --:--:-- 8909k


In [28]:
df1 = pd.read_csv("./aqi-2022-2024.csv")
df2 = pd.read_csv("./aqi-2019-2021.csv")
df = pd.concat([df1, df2])
df.rename(columns={
    'polluant': 'pollutant',
    'valeur': 'value',
    'date': 'date',
    'heure': 'hour'
}, inplace=True)
# Creating a datetime column for easier manipulation later on
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hour'].astype(str) + ':00:00',
                                format = '%Y-%m-%d %H:%M:%S',
                                errors = 'coerce')
# Dropping columns we won't need anymore
df.drop(["hour", "pollutant", "date"], axis=1, inplace=True)

# Indexing by station and datetime and grab the maximum value for each index (related to our assumption earlier)
df = df.groupby(['stationId', 'datetime']).max("value")
df = df.groupby("datetime").mean("value").reset_index()
# Sorting the dataframe by datetime for better visualization
df.sort_values("datetime", inplace=True)

In [29]:
import pandas as pd
import concurrent.futures

station_id = "30165"
years = range(2019, 2025)
months = range(1, 13)
timeframe = "1"
dates = [(month, year) for year in years for month in months]

def download_data(date):
    month, year = date
    url = f"https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station_id}&Year={year}&Month={month}&Day=1&timeframe={timeframe}&submit=Download+Data"
    return pd.read_csv(url)

with concurrent.futures.ThreadPoolExecutor() as executor:
    all_files = executor.map(download_data, dates)
   
weather_df = pd.concat(all_files)
weather_df.rename(columns={'Date/Time (LST)': 'datetime', "Temp (°C)": "temp", "Precip. Amount (mm)": "precip", "Rel Hum (%)": "rel_humid"}, inplace=True)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
weather_df = weather_df[['datetime', "temp", "rel_humid", "precip"]]

weather_df

,datetime,temp,rel_humid,precip
0,2019-01-01 00:00:00,1.9,95.0,1.7
1,2019-01-01 01:00:00,1.8,93.0,0.7
2,2019-01-01 02:00:00,2.1,95.0,3.0
3,2019-01-01 03:00:00,2.2,95.0,4.5
4,2019-01-01 04:00:00,2.3,96.0,3.8
...,...,...,...,...
739,2024-12-31 19:00:00,NaN,NaN,NaN
740,2024-12-31 20:00:00,NaN,NaN,NaN
741,2024-12-31 21:00:00,NaN,NaN,NaN
742,2024-12-31 22:00:00,NaN,NaN,NaN


In [30]:
merged_df = pd.merge(df, weather_df, on="datetime", how="left")
merged_df['year'] = merged_df['datetime'].dt.year
merged_df['year'] = merged_df['year'] / merged_df['year'].max()
merged_df['month'] = merged_df['datetime'].dt.month
merged_df['day'] = merged_df['datetime'].dt.day
merged_df['hour'] = merged_df['datetime'].dt.hour
merged_df['weekday'] = merged_df['datetime'].dt.weekday
merged_df['value'] = merged_df['value'].clip(upper=100)
max_value = merged_df['value'].max()
merged_df['value'] = merged_df['value'] / max_value
merged_df.fillna({"precip": 0,
                   "temp": merged_df['temp'].fillna(method='bfill'),
                   "rel_humid": merged_df['rel_humid'].mode()[0]}, inplace=True)
merged_df

/var/folders/gd/ctbm7cmd6n124vc1h8766rlw0000gn/T/ipykernel_24506/1605622287.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  "temp": merged_df['temp'].fillna(method='bfill'),


,datetime,value,temp,rel_humid,precip,year,month,day,hour,weekday
0,2019-01-01 00:00:00,0.334545,1.9,95.0,1.7,0.99753,1,1,0,1
1,2019-01-01 01:00:00,0.281818,1.8,93.0,0.7,0.99753,1,1,1,1
2,2019-01-01 02:00:00,0.193636,2.1,95.0,3.0,0.99753,1,1,2,1
3,2019-01-01 03:00:00,0.134545,2.2,95.0,4.5,0.99753,1,1,3,1
4,2019-01-01 04:00:00,0.109091,2.3,96.0,3.8,0.99753,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...
48067,2024-06-25 19:00:00,0.271000,25.2,46.0,0.0,1.00000,6,25,19,1
48068,2024-06-25 20:00:00,0.261000,23.8,58.0,0.0,1.00000,6,25,20,1
48069,2024-06-25 21:00:00,0.239000,23.6,59.0,0.0,1.00000,6,25,21,1
48070,2024-06-25 22:00:00,0.245000,19.9,84.0,0.0,1.00000,6,25,22,1


In [31]:
import numpy as np
from fastai.tabular.all import *

date_valid = pd.Timestamp('2023-01-01')
date_test = pd.Timestamp('2024-01-01')
now = pd.Timestamp('2024-06-01')

# From Jan 01, 2019 -> Dec 31, 2022
train_idx = merged_df['datetime'] < date_valid

# From Jan 01, 2023 -> Dec 31, 2023
valid_idx = (merged_df['datetime'] >= date_valid) & (merged_df['datetime'] < date_test)

# From Jan 01, 2024 -> June 01, 2024
test_idx = (merged_df['datetime'] >= date_test) & (merged_df['datetime'] < now)

train_idxs = np.where(train_idx)[0].tolist()
valid_idxs = np.where(valid_idx)[0].tolist()
test_idxs = np.where(test_idx)[0].tolist()

cont,cat = cont_cat_split(merged_df, max_card=20, dep_var='value')
dls = TabularPandas(
  merged_df,
  procs=[Categorify, Normalize],
  cat_names=cat,
  cont_names=cont,
  y_names=['value'],
  splits=(train_idxs, valid_idxs),
  y_block=RegressionBlock()
).dataloaders(bs=2048)

In [32]:
learn = tabular_learner(dls, metrics=mae, layers=[250, 100], y_range=(0, 1))
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,mae,time
0,0.138118,0.071449,0.259237,00:00
1,0.126728,0.071438,0.254919,00:00
2,0.110009,0.093728,0.286704,00:00
3,0.091794,0.076902,0.249465,00:00
4,0.075047,0.057618,0.200717,00:00
5,0.060928,0.046950,0.167216,00:00
6,0.049792,0.042691,0.153482,00:00
7,0.041468,0.041640,0.145898,00:00
8,0.035485,0.041033,0.145078,00:00
9,0.031253,0.041656,0.146126,00:00


In [33]:
df_test = merged_df.loc[test_idxs]
test_dl = dls.test_dl(df_test)

preds, targets = learn.get_preds(dl=test_dl)

mae(preds, targets)

TensorBase(0.2495)

In [34]:
def mk_analysis_df(df, preds):
  # Create DataFrame
  _df = pd.DataFrame({
      'datetime': df['datetime'],
      'Actual': df['value'] * max_value,
      'Predicted': preds.flatten() * max_value,
  })
  
  
  _df['Error'] = abs((_df['Actual'] - _df['Predicted']) / _df['Actual']) * 100


  return _df

In [35]:
export_df = mk_analysis_df(df_test, preds.flatten())
export_df.rename(columns={'datetime': 'date'}, inplace=True)

mean_error = export_df['Error'].mean()

json_output = export_df.to_json(orient='records', indent=2)
json_dict = json.loads(json_output)

# Create a new dictionary with the "results" property
result_dict = {"mean_error": mean_error, "results": json_dict}

# Convert dictionary to JSON string with indentation
json.dumps(result_dict)


'{"mean_error": 153.57933550541063, "results": [{"date": 1704067200000, "Actual": 15.7272727273, "Predicted": 33.9958229065, "Error": 116.1584115442}, {"date": 1704070800000, "Actual": 16.2727272727, "Predicted": 33.2849845886, "Error": 104.5445980306}, {"date": 1704074400000, "Actual": 17.0909090909, "Predicted": 32.5719261169, "Error": 90.5804187693}, {"date": 1704078000000, "Actual": 17.6363636364, "Predicted": 32.1839447021, "Error": 82.4862843936}, {"date": 1704081600000, "Actual": 18.2727272727, "Predicted": 31.9055175781, "Error": 74.607310129}, {"date": 1704085200000, "Actual": 18.2727272727, "Predicted": 31.7841377258, "Error": 73.9430422807}, {"date": 1704088800000, "Actual": 18.3636363636, "Predicted": 32.0669670105, "Error": 74.6220975819}, {"date": 1704092400000, "Actual": 16.8181818182, "Predicted": 35.6623191833, "Error": 112.0462221713}, {"date": 1704096000000, "Actual": 15.5454545455, "Predicted": 37.3688278198, "Error": 140.3842725252}, {"date": 1704099600000, "Actual